In [2]:
!pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2426327aeb87e18cc8bd4b3eb32daba055f6b87f19a9dd3f2eab1648389c78fc
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [32]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/file_moi.csv")
dataset = Dataset.from_pandas(df)

# Chia train-test (80%-20%)
dataset = dataset.train_test_split(test_size=0.2)

ParserError: Error tokenizing data. C error: Expected 5 fields in line 28, saw 6


In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

model_name = "vinai/bartpho-syllable-base"  # Hoặc "vinai/bartpho-word"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bartpho_summarization",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,  # Training nhiều epoch nhưng có early stopping
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    report_to="none",
    fp16=True,
    weight_decay=0.01,  # Regularization
    learning_rate=5e-5,  # Giảm learning rate để tránh overfitting
    lr_scheduler_type="linear",

    load_best_model_at_end=True,  # ✅ Cần có để dùng EarlyStopping
    metric_for_best_model="eval_loss",  # Chọn tiêu chí tốt nhất
    greater_is_better=False,  # Loss càng nhỏ càng tốt
)


model.config.dropout = 0.3
model.config.attention_dropout = 0.3



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
def preprocess_function(examples):
    inputs = examples["Nội dung"]  # Dữ liệu đã là str
    targets = examples["Tóm tắt"]

    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    labels = tokenizer(
        targets,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Chuyển token padding thành -100 để Hugging Face bỏ qua khi tính loss
    model_inputs["labels"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    return model_inputs

# Áp dụng preprocessing lên dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["Số hiệu", "Tiêu đề", "Ngày ban hành"])


Map:   0%|          | 0/988 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer.train()

<ipython-input-28-4280da9f2489>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.429500,0.752104
2,0.325000,0.780221
3,0.281900,0.788977
4,0.238400,0.785748
5,0.379600,0.762156
6,0.324200,0.769461


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=744, training_loss=0.33726916902808735, metrics={'train_runtime': 78.0705, 'train_samples_per_second': 1265.523, 'train_steps_per_second': 158.831, 'total_flos': 1807314805850112.0, 'train_loss': 0.33726916902808735, 'epoch': 6.0})

In [30]:
import numpy as np
import torch
import evaluate  # Dùng evaluate thay vì load_metric

def evaluate_model(model, tokenizer, test_texts, test_summaries=None, max_input_length=512, max_output_length=128):
    """
    Đánh giá mô hình trên tập dữ liệu mới.

    Args:
        model: Mô hình đã train.
        tokenizer: Tokenizer tương ứng.
        test_texts: Danh sách văn bản đầu vào.
        test_summaries: Danh sách tóm tắt thực tế (nếu có) để tính ROUGE.
        max_input_length: Độ dài tối đa của văn bản đầu vào.
        max_output_length: Độ dài tối đa của bản tóm tắt đầu ra.

    Returns:
        Nếu có `test_summaries`, trả về điểm ROUGE. Nếu không, chỉ trả về bản tóm tắt mô hình sinh ra.
    """

    model.eval()  # Đưa mô hình vào chế độ đánh giá (tắt dropout)
    predictions = []

    for text in test_texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=max_input_length)
        input_ids = inputs.input_ids.to(model.device)

        with torch.no_grad():
            output_ids = model.generate(input_ids, max_length=max_output_length, num_beams=5, early_stopping=True)

        # Giải mã văn bản
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predictions.append(summary)

    if test_summaries:
        # Tính điểm ROUGE
        rouge_metric = evaluate.load("rouge")
        rouge_scores = rouge_metric.compute(predictions=predictions, references=test_summaries, use_stemmer=True)

        return predictions, rouge_scores
    else:
        return predictions
